In [26]:
## Load required libraries
import requests
import shutil
from datetime import datetime
from pathlib import Path
import zipfile
import geopandas as gpd
import os
import pandas as pd

1 Request data from EFFIS database 

In [4]:
# Download data from EFFIS database

# Specify the URL
url = "https://maps.wild-fire.eu/effis?service=WFS&request=getfeature&typename=ms:modis.ba.poly&version=1.1.0&outputformat=SHAPEZIP"

# Specify the folder to save the downloaded file
input_folder = Path("L:/f02_data/wildfires/spatial_data/input")

# Create the folder if it doesn't exist
input_folder.mkdir(parents=True, exist_ok=True)

# Get the current date and time
current_datetime = datetime.now().strftime("%Y%m%d")

# Make the GET request
response = requests.get(url, stream=True)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Specify the output file path with the current date and time
    output_file_path = input_folder / f"effis_rawdata_{current_datetime}.zip"

    # Save the content to a file
    with open(output_file_path, 'wb') as f:
        shutil.copyfileobj(response.raw, f)

    # Print a message indicating success
    print(f"Dataset downloaded and saved to {output_file_path}")
else:
    # Print an error message if the request was not successful
    print(f"Error: Unable to download dataset. Status code {response.status_code}")

Dataset downloaded and saved to L:\f02_data\wildfires\spatial_data\input\effis_rawdata_20240109.zip


2 Prepare data for processing

In [9]:
# Unzip

# Specify the path to the downloaded ZIP file
zip_file_path = input_folder / f"effis_rawdata_{current_datetime}.zip"

# Specify the folder where you want to extract the contents
extracted_folder = input_folder / f"effis_rawdata_{current_datetime}"

# Create the folder if it doesn't exist
extracted_folder.mkdir(parents=True, exist_ok=True)

# Open the ZIP file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Print a message indicating success
print(f"ZIP file extracted to {extracted_folder}")

ZIP file extracted to L:\f02_data\wildfires\spatial_data\input\effis_rawdata_20240109


In [27]:
# Load the shapefile into a GeoDataFrame
shapefile_path = os.path.join(extracted_folder, 'modis.ba.poly.shp') 
gdf = gpd.read_file(shapefile_path)

# Convert the 'date_column' to pandas datetime object
gdf['FIREDATE'] = pd.to_datetime(gdf['FIREDATE'])